## Aufgabe 32: $\chi^2$-Test

Für den $\chi^2$-Test wird jeweils folgende Größe berechnet:
$$
\chi^2 = \sum \frac{(E_i - E_H)^2}{\sigma^2} 
$$
$E_H$ bezeichnet den Energiewert der Hypothese. Die Standardabweichung $\sigma$ muss aus den Daten geschätzt werden. Hierfür wird $\sigma = 0.5$ angenommen.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
E = np.array([31.6, 31.3, 32.2, 30.8, 31.2, 31.3, 31.9])

__a)__

In [ ]:
E_A = 31.3
sig = 0.5

chi2_A = np.sum( ( E - E_A )**2 / sig**2 )
chi2_A

__b)__

In [ ]:
E_B = 30.7
chi2_B = np.sum( ( E - E_B )**2 / sig**2 )
chi2_B

In [ ]:
plt.axhline(y = E_A, label = 'Hypothese A', color = 'g')
plt.axhline(y = E_B, label = 'Hypothese B', color = 'r')
plt.errorbar(x = np.arange(7), y = E,yerr = 0.5, 
             label = 'Messwerte', 
             linestyle = '', marker = 'o', color = 'k')
plt.ylabel('Energie / meV')
plt.legend()
plt.xticks([])
None

In $\chi^2$-Tabelle findet man bei $7 - 1 = 6$ Freiheitsgraden für 5% Signifikanz den Wert $12,59$. Eine Hypothese wird daher abgelehnt, wenn $\chi^2 > 12,59$ ist. Daher kann Hypothese B verworfen werden. Hypothese A kann auf Grundlage der gefundenen Daten nicht verworfen werden. 

Es wurde die Varianz der theoretischen Verteilung geschätzt, daher muss die Zahl der Freiheitsgrade um 1 verringert werden. Hier läge der $\chi^2$-Wert bei $11,07$. Es ändert sich somit nichts am Ergebnis.

## Aufgabe 33

__a)__ Wähle $\mu = \lambda$ und $\sigma = \sqrt{\lambda}$

__b)__ + __c)__ + __d)__ Da die Bestimmung des Wertes für $\lambda$ durch einen Zufallsprozess erfolgt, wird der Mittelwert aus 100 Durchläufen verwendet.

In [ ]:
for alpha in [0.05, 0.025, 0.001]:
    print(f'Für die Signifikanz alpha = {alpha}')
    K_alpha = np.sqrt(np.log(2 / alpha) / 2 )

    lambda_accepted_list = []
    for i in range(100):
        for lam in np.arange(1, 51):
            #sample gauss and poisson distribution
            gaus_sample = np.round(np.random.normal(lam, np.sqrt(lam),
                                                    size=10000), 0)
            
            poisson_sample = np.random.poisson(lam, size = 10000)
            
            
            #binning
            gaus_hist, gaus_edges = np.histogram(gaus_sample, bins = 100, 
                                         range= (lam - 5 * np.sqrt(lam), 
                                                 lam + 5 * np.sqrt(lam)))
            
            poisson_hist, poisson_edges = np.histogram(poisson_sample, bins = 100, 
                                         range= (lam - 5 * np.sqrt(lam), 
                                                 lam + 5 * np.sqrt(lam))) 
                
                
            #create the cdfs    
            gaus_cdf = np.array([np.sum(gaus_hist[:i + 1]) / len(gaus_sample) 
                                 for i in range(100)])
            poisson_cdf = np.array([np.sum(poisson_hist[:i + 1]) / len(poisson_sample) 
                                    for i in range(100)])
            
            
            #calculate the max distance between cdfs
            d_max = np.max(np.abs(gaus_cdf - poisson_cdf))
            
            
            #test condition
            lambda_accepted = np.sqrt(len(gaus_sample)/2) * d_max < K_alpha
            if lambda_accepted:
                lambda_accepted_list.append(lam)
                break
                
    print(f'kann ab lambda = {round(np.mean(lambda_accepted_list), 0)} +/- {round(np.std(lambda_accepted_list), 0)}... \n')
print('...die Hypothese, dass beide Verteilungen gleich sind, nicht mehr verworfen werden.')

## Aufgabe 34: Ballon-Experiment

__a)__ Berechnungen/Herleitungen handschriftlich.

In [ ]:
counts = np.array([4135, 4202, 4203, 4218, 4227, 4231, 4310])
days = np.arange(1, 8)

Wahrscheinlichste Zählrate entspricht dem Mittelwert

In [ ]:
c_mean = round(np.mean(counts), 0)
print(c_mean)

__b)__ Bestimme Parameter numerisch

In [ ]:
from scipy.optimize import fmin

In [ ]:
def llh(params):
    A = params[0]
    B = params[1]
    return + len(counts) * B + A * np.sum(days) - np.sum(counts * np.log(A * days + B))

In [ ]:
lin_model = fmin(llh, [0, 4000]) 
print(f'Parameter für lineares Modell: {lin_model}')

In [ ]:
plt.plot(days, counts, 'ko', label = 'Daten')
plt.axhline(y = np.mean(counts), label = 'Konstantes Modell', color = 'g')
plt.plot(days, lin_model[0] * days + lin_model[1], 
         color = 'r', label = 'Lineares Modell') 
plt.xlabel('Tag')
plt.ylabel('Counts')
plt.legend()
None

__c)__ Test ist der Quotient aus den einzelnen Likelihoods.

In [ ]:
gamma = np.prod((c_mean / (lin_model[0] * days + lin_model[1]))**counts 
                * np.exp(-c_mean + lin_model[0] * days + lin_model[1]))

Aus $\Gamma$ kann die $\chi^2$ verteilte Größe $-2ln(\Gamma)$ berechnet werden.

In [ ]:
- 2 * np.log(gamma)

Aus Vergleich mit einer Tabelle ergibt dies eine Signifikanz von etwa 5%. 

__d)__ Analoges Vorgehen:

In [ ]:
counts_d = np.array([4135, 4202, 4203, 4218, 4227, 4231, 4310, 4402])
days_d = np.append(days, 14)

In [ ]:
c_mean_d = round(np.mean(counts_d), 0)

In [ ]:
def llh(params):
    A = params[0]
    B = params[1]
    return + len(counts_d) * B + A * np.sum(days_d) - np.sum(counts_d * np.log(A * days_d + B))

In [ ]:
lin_model_d = fmin(llh, [0, counts_d[0]]) 
print(f'Parameter für lineares Modell: {lin_model_d}')

In [ ]:
plt.plot(days_d, counts_d, 'ko', label = 'Daten')
plt.axhline(y = c_mean_d, label = 'Konstantes Modell', color = 'g')
plt.plot(days_d, lin_model_d[0] * days_d + lin_model_d[1], 
         color = 'r', label = 'Lineares Modell') 
plt.xlabel('Tag')
plt.ylabel('Counts')
plt.legend()
None

In [ ]:
gamma_d = np.prod((c_mean_d / (lin_model_d[0] * days_d + lin_model_d[1]))**counts_d * np.exp(-c_mean_d + lin_model_d[0] * days_d + lin_model_d[1]))

In [ ]:
- 2 * np.log(gamma_d)

Aus Vergleich mit einer Tabelle ergibt dies eine Signifikanz von etwa 0.1%. 

## Aufgabe 36: Zwei Histogramme

Berechnungen handschriftlich.

__e)__  

In [ ]:
n = np.array([111, 188, 333])
m = np.array([15, 36, 30])
N = np.sum(n)
M = np.sum(m)

In [ ]:
chi2 = 1 / N / M * np.sum((N * m - M * n)**2 / (n + m))

In [ ]:
chi2